In [64]:
import pandas as pd
import geopandas as gpd
import os
from mpilot.program import Program
import arcpy
from shapely import wkt
import zipfile

THIS_FOLDER = os.path.dirname(os.path.abspath("RPP-SBC-EEMS"))

In [65]:
arcpy.ResetEnvironments()
gdb_name = "EEMS_Output.gdb"
arcpy.env.overwriteOutput = True
path = os.path.join(THIS_FOLDER, gdb_name)

if not arcpy.Exists(path):
    arcpy.CreateFileGDB_management("./", gdb_name)

arcpy.env.workspace = path
arcpy.env.scratchWorkspace = path

In [66]:
def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), os.path.join(path, '..')))

In [67]:
def eems_read(p, r):
    EEMSRead = p.find_command_class('EEMSRead')
    p.add_command(EEMSRead, 'Join_ID', {'InFileName': r, 
        'InFieldName': 'Join_ID',
        'Metadata': {'DisplayName' : 'Join+ID'}})

    p.add_command(EEMSRead, 'Mj_LU_Cat', {'InFileName': r, 
        'InFieldName': 'Mj_LU_Cat',
        'Metadata': {'DisplayName' : 'Majority+Land+Use+Category+Raw',
                     'Description' : 'Majority+Land+Use+Category+Raw',
                     'ColorMap' : 'RdYlBu_r'}})

    p.add_command(EEMSRead, 'Mj_Haz_Cat', {'InFileName': r, 
        'InFieldName': 'Mj_Haz_Cat',
        'Metadata': {'DisplayName' : 'Fire+Hazard+Severity+Raw',
                     'Description' : 'Fire+Hazard+Severity+Raw',
                     'ColorMap' : 'RdYlBu_r'}})

    p.add_command(EEMSRead, 'Pr_Lg_Fr', {'InFileName': r, 
        'InFieldName': 'Pr_Lg_Fr',
        'Metadata': {'DisplayName' : 'Probability+Of+Large+Fire+Raw',
                     'Description' : 'Probability+Of+Large+Fire+Raw',
                     'ColorMap' : 'RdYlBu_r'}})

    p.add_command(EEMSRead, 'Pr_Fi_Ign', {'InFileName': r, 
        'InFieldName': 'Pr_Fi_Ign',
        'Metadata': {'DisplayName' : 'Probability+Of+Fire+Ignition+Raw',
                     'Description' : 'Probability+Of+Fire+Ignition+Raw',
                     'ColorMap' : 'RdYlBu_r'}})

    p.add_command(EEMSRead, 'Pc_Cg_LgFi', {'InFileName': r, 
        'InFieldName': 'Pc_Cg_LgFi',
        'Metadata': {'DisplayName' : 'Average+Percent+Change+In+Large+Fire+Probability+Raw',
                     'Description' : 'Average+Percent+Change+In+Large+Fire+Probability+Raw',
                     'ColorMap' : 'RdYlBu_r'}})

    p.add_command(EEMSRead, 'Fl_Fa', {'InFileName': r, 
        'InFieldName': 'Fl_Fa',
        'Metadata': {'DisplayName' : 'Flora+And+Fauna+Raw',
                     'Description' : 'Flora+And+Fauna+Raw',
                     'ColorMap' : 'RdYlBu_r'}})

    p.add_command(EEMSRead, 'WUI_Interm', {'InFileName': r, 
        'InFieldName': 'WUI_Interm',
        'Metadata': {'DisplayName' : 'WUI+Intermix',
                     'Description' : 'WUI+Intermix',
                     'ColorMap' : 'RdYlBu_r'}})

    p.add_command(EEMSRead, 'WUI_Interf', {'InFileName': r, 
        'InFieldName': 'WUI_Interf',
        'Metadata': {'DisplayName' : 'WUI+Interface',
                     'Description' : 'WUI+Interface',
                     'ColorMap' : 'RdYlBu_r'}})
    
    p.add_command(EEMSRead, 'Buil_Rat', {'InFileName': r, 
        'InFieldName': 'Buil_Rat',
        'Metadata': {'DisplayName' : 'Building+Site+Ratio',
                     'Description' : 'Building+Site+Ratio',
                     'ColorMap' : 'RdYlBu_r'}})
    p.add_command(EEMSRead, 'Vg_Pct', {'InFileName': r, 
        'InFieldName': 'Vg_Pct',
        'Metadata': {'DisplayName' : 'Vegetation+Percentage',
                     'Description' : 'Vegetation+Percentage',
                     'ColorMap' : 'RdYlBu_r'}})
    p.add_command(EEMSRead, 'Slp', {'InFileName': r, 
        'InFieldName': 'Slp',
        'Metadata': {'DisplayName' : 'Slope+Raw',
                     'Description' : 'Slope+Raw',
                     'ColorMap' : 'RdYlBu_r'}})
    

In [68]:
def eems_normalize(p, r):
    Normalize = p.find_command_class('Normalize')
    NormalizeCat = p.find_command_class('NormalizeCat')
    NormalizeCurve = p.find_command_class('NormalizeCurve')
    MeanToMid = p.find_command_class('NormalizeMeanToMid')
    p.add_command(NormalizeCat, "nm_lu_cat", {'InFieldName': 'Mj_LU_Cat', 
        'RawValues': [0, 1, 2, 3, 4],
        'NormalValues' : [1, 0, 0.1, 0, 0],
        'DefaultNormalValue' : 0,                               
        'Metadata': {'DisplayName' : 'Not+Land+Type+Buffer'}})
    p.add_command(NormalizeCat, "nm_hz_cat", {'InFieldName': 'Mj_Haz_Cat', 
        'RawValues': [3, 2, 1, 0, -2, -1],
        'NormalValues' : [1, 0.75, 0.5, 0, 0, 0],
        'DefaultNormalValue' : 0,                               
        'Metadata': {'DisplayName' : 'Fire+Hazard+Severity+Zone'}})
    p.add_command(Normalize, "Nm_Fl_Fa", {'InFieldName': 'Fl_Fa', 
        'StartVal': 1,
        'EndVal' : 0,
        'Metadata': {'DisplayName' : 'Low+Ecological+Importance'}})
    p.add_command(Normalize, "Nm_Lg_Fire", {'InFieldName': 'Pr_Lg_Fr', 
        'StartVal': 0,
        'EndVal' : 1,
        'Metadata': {'DisplayName' : 'Probability+Of+Large+Fire'}})
    p.add_command(Normalize, "Nm_Pr_F_Ig", {'InFieldName': 'Pr_Fi_Ign', 
        'StartVal': 0,
        'EndVal' : 1,
        'Metadata': {'DisplayName' : 'Probability+Of+Large+Fire'}})
    p.add_command(Normalize, "Nm_PcC_LgF", {'InFieldName': 'Pc_Cg_LgFi', 
        'StartVal': 0,
        'EndVal' : 1,
        'Metadata': {'DisplayName' : 'Percent+Change+In+Large+Fire+Probability'}})
    p.add_command(MeanToMid, "NmBuil_Rat", {'InFieldName': 'Buil_Rat', 
        'IgnoreZeros': False, 
        'NormalValues': [1, 0.75, 0.5, 0.25, 0],
        'Metadata': {'DisplayName' : 'Undevelopment+Index'}})
    p.add_command(NormalizeCurve, "Nm_Slp", {'InFieldName': "Slp",
        'RawValues': [0, 6, 16],
        'NormalValues' : [0, 1, 0],
        'Metadata': {'DisplayName' : 'Steeper+Slope+Index'}})

In [69]:
def eems_operations_a(p, r):
    WeightedSum = p.find_command_class('WeightedSum')
    Normalize = p.find_command_class('Normalize')
    Maximum = p.find_command_class('Maximum')
    
    p.add_command(WeightedSum, 'C_Fr_Rsk', {'InFieldNames': ['Nm_Lg_Fire', 'Nm_Pr_F_Ig', 'Nm_PcC_LgF'], 
        'Weights' : [0.6, 0.2, 0.2],
        'Metadata': {'DisplayName' : 'CBI+Fire+Risk+Raw'}})
    
    p.add_command(Normalize, "CNm_Fr_Rsk", {'InFieldName': 'C_Fr_Rsk', 
        'StartVal': 0,
        'EndVal' : 1,
        'Metadata': {'DisplayName' : 'CBI+Fire+Risk+Index'}})
    
    p.add_command(WeightedSum, 'Fr_Rsk', {'InFieldNames': ['CNm_Fr_Rsk', 'nm_hz_cat'], 
        'Weights' : [0.9, 0.1],
        'Metadata': {'DisplayName' : 'Fire+Risk+Raw'}})
    
    p.add_command(Normalize, "Nm_Fr_Rsk", {'InFieldName': 'Fr_Rsk', 
        'StartVal': 0,
        'EndVal' : 1,
        'Metadata': {'DisplayName' : 'Fire+Risk+Index'}})
    
    p.add_command(Maximum, "WUI_Rsk", {'InFieldNames': ['WUI_Interm', 'WUI_Interf'], 
        'Metadata': {'DisplayName' : 'WUI+Risk'}})
    
    p.add_command(WeightedSum, 'Hi_Pri', {'InFieldNames': ['Nm_Fr_Rsk', 'Nm_Fl_Fa', 'nm_lu_cat', 'WUI_Rsk'], 
        'Weights' : [0.3, 0.2, 0.25, 0.25],
        'Metadata': {'DisplayName' : 'High+Priority+Site+Raw'}})
    
    p.add_command(Normalize, "Nm_Hi_Pri", {'InFieldName': 'Hi_Pri', 
        'StartVal': 0,
        'EndVal' : 1,
        'Metadata': {'DisplayName' : 'High+Priority+Site'}})

In [70]:
def eems_operations_b(p, r):
    Multiply = p.find_command_class('Multiply')
    Normalize = p.find_command_class('Normalize')
    p.add_command(Multiply, 'op_sp_re', {'InFieldNames': ['Nm_Hi_Pri', 'NmBuil_Rat'], 
        'Metadata': {'DisplayName' : 'Openspace+Wildfire+Risk+Reduction+Priority+Areas+Raw'}})
    
    p.add_command(Normalize, "os_hi_pri", {'InFieldName': 'op_sp_re', 
        'StartVal': 0,
        'EndVal' : 1,
        'Metadata': {'DisplayName' : 'Openspace+Wildfire+Risk+Reduction+Priority+Areas'}})

In [71]:
def eems_operations_c(p, r):
    Multiply = p.find_command_class('Multiply')
    Normalize = p.find_command_class('Normalize')
    p.add_command(Multiply, 'gt_idx_rw', {'InFieldNames': ["Vg_Pct", "Nm_Slp"], 
        'Metadata': {'DisplayName' : 'Herbivory+Index+Raw'}})
    
    p.add_command(Normalize, "gt_idx", {'InFieldName': 'gt_idx_rw', 
        'StartVal': 0,
        'EndVal' : 1,
        'Metadata': {'DisplayName' : 'Herbivory+Index'}})
    
    p.add_command(Multiply, 'gt_wfra', {'InFieldNames': ['os_hi_pri', 'gt_idx'], 
        'Metadata': {'DisplayName' : 'Herbivory+Wildfire+Risk+Reduction+Priority+Areas+Raw'}})
    p.add_command(Normalize, "nmgt_wfra", {'InFieldName': 'gt_wfra', 
        'StartVal': 0,
        'EndVal' : 1,
        'Metadata': {'DisplayName' : 'Herbivory+Wildfire+Risk+Reduction+Priority+Areas'}})
    

In [72]:
def eems_out(p, o):
    EEMSWrite = p.find_command_class('EEMSWrite')
    p.add_command(EEMSWrite, 'out', {'OutFileName': o, 'OutFieldNames': list(p.commands.keys())})

### Model A - Wildfire Risk Reduction Areas

In [82]:
reporting_units_path = os.path.join(THIS_FOLDER, "out_data/eems/Reporting_Units.csv")
output_path = os.path.join(THIS_FOLDER, "out_data/eems/EEMS_Output_A.csv")

p = Program()
eems_read(p, reporting_units_path)
eems_normalize(p, reporting_units_path)
eems_operations_a(p, reporting_units_path)
eems_out(p, output_path)

p.run()

out_file = "out_data/eems/EEMS_Model_A.shp"
eems = pd.merge(pd.read_csv(output_path), pd.read_csv(reporting_units_path)[["Join_ID", "geometry"]], on='Join_ID', how='inner')
eems['geometry'] = eems['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(eems, crs = "EPSG:26911", geometry = eems.geometry)
gdf.to_file(filename = out_file)

path = os.path.join(THIS_FOLDER, out_file)
arcpy.CopyFeatures_management(path, "EEMS_Model_A")

del p.commands['Join_ID']
del p.commands['out']
del p.commands['Vg_Pct']
del p.commands['Buil_Rat']
del p.commands['Slp']
del p.commands['NmBuil_Rat']
del p.commands['Nm_Slp']

p.to_file('EEMS_CMD_A1.mpt')

f1 = open("EEMS_CMD_A1.mpt", 'r')
f2 = open("EEMS_CMD_A2.mpt", 'w')
for line in f1:
    f2.write(line.replace('"', ''))
f1.close()
f2.close()

f1 = open("EEMS_CMD_A2.mpt", 'r')
f2 = open("EEMS_CMD_A3.mpt", 'w')
for line in f1:
    f2.write(line.replace("'", ''))
f1.close()
f2.close()

f1 = open("EEMS_CMD_A3.mpt", 'r')
f2 = open("EEMS_CMD_A.mpt", 'w')
for line in f1:
    f2.write(line.replace("+", '&nbsp;'))
f1.close()
f2.close()

os.remove('EEMS_CMD_A1.mpt')
os.remove('EEMS_CMD_A2.mpt')
os.remove('EEMS_CMD_A3.mpt')

gdb_name = "EEMS_Model_A.gdb"
path = os.path.join(THIS_FOLDER, gdb_name)


try:
    arcpy.CreateFileGDB_management("./", gdb_name)
except:
    print("{} already exists".format(gdb_name))
    pass

out_featureclass = os.path.join(path, "EEMS_Model_A")
arcpy.CopyFeatures_management("EEMS_Model_A", out_featureclass)

zipf = zipfile.ZipFile('EEMS_Model_A.gdb.zip', 'w', zipfile.ZIP_DEFLATED)
zipdir("EEMS_Model_A.gdb/", zipf)
zipf.close()

### Model B - Open-space Wildfire Risk Reduction Priority Areas

In [83]:
reporting_units_path = os.path.join(THIS_FOLDER, "out_data/eems/Reporting_Units.csv")
output_path = os.path.join(THIS_FOLDER, "out_data/eems/EEMS_Output_B.csv")

p = Program()
eems_read(p, reporting_units_path)
eems_normalize(p, reporting_units_path)
eems_operations_a(p, reporting_units_path)
eems_operations_b(p, reporting_units_path)
eems_out(p, output_path)
p.run()

out_file = "out_data/eems/EEMS_Model_B.shp"
eems = pd.merge(pd.read_csv(output_path), pd.read_csv(reporting_units_path)[["Join_ID", "geometry"]], on='Join_ID', how='inner')
eems['geometry'] = eems['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(eems, crs = "EPSG:26911", geometry = eems.geometry)
gdf.to_file(filename = out_file)

path = os.path.join(THIS_FOLDER, out_file)
arcpy.CopyFeatures_management(path, "EEMS_Model_B")

del p.commands['Join_ID']
del p.commands['out']
del p.commands['Vg_Pct']
del p.commands['Slp']
del p.commands['Nm_Slp']

p.to_file('EEMS_CMD_B1.mpt')

f1 = open("EEMS_CMD_B1.mpt", 'r')
f2 = open("EEMS_CMD_B2.mpt", 'w')
for line in f1:
    f2.write(line.replace('"', ''))
f1.close()
f2.close()

f1 = open("EEMS_CMD_B2.mpt", 'r')
f2 = open("EEMS_CMD_B3.mpt", 'w')
for line in f1:
    f2.write(line.replace("'", ''))
f1.close()
f2.close()

f1 = open("EEMS_CMD_B3.mpt", 'r')
f2 = open("EEMS_CMD_B.mpt", 'w')
for line in f1:
    f2.write(line.replace("+", '&nbsp;'))
f1.close()
f2.close()

os.remove('EEMS_CMD_B1.mpt')
os.remove('EEMS_CMD_B2.mpt')
os.remove('EEMS_CMD_B3.mpt')

gdb_name = "EEMS_Model_B.gdb"
path = os.path.join(THIS_FOLDER, gdb_name)

try:
    arcpy.CreateFileGDB_management("./", gdb_name)
except:
    print("{} already exists".format(gdb_name))
    pass

out_featureclass = os.path.join(path, "EEMS_Model_B")
arcpy.CopyFeatures_management("EEMS_Model_B", out_featureclass)

zipf = zipfile.ZipFile('EEMS_Model_B.gdb.zip', 'w', zipfile.ZIP_DEFLATED)
zipdir("EEMS_Model_B.gdb/", zipf)
zipf.close()

### Model C - Prescribed Herbivory Priority Areas

In [84]:
reporting_units_path = os.path.join(THIS_FOLDER, "out_data/eems/Reporting_Units.csv")
output_path = os.path.join(THIS_FOLDER, "out_data/eems/EEMS_Output_C.csv")

p = Program()
eems_read(p, reporting_units_path)
eems_normalize(p, reporting_units_path)
eems_operations_a(p, reporting_units_path)
eems_operations_b(p, reporting_units_path)
eems_operations_c(p, reporting_units_path)
eems_out(p, output_path)
p.run()

out_file = "out_data/eems/EEMS_Model_C.shp"
eems = pd.merge(pd.read_csv(output_path), pd.read_csv(reporting_units_path)[["Join_ID", "geometry"]], on='Join_ID', how='inner')
eems['geometry'] = eems['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(eems, crs = "EPSG:26911", geometry = eems.geometry)
gdf.to_file(filename = out_file)

path = os.path.join(THIS_FOLDER, out_file)
arcpy.CopyFeatures_management(path, "EEMS_Model_C")

del p.commands['Join_ID']
del p.commands['out']

p.to_file('EEMS_CMD_C1.mpt')

f1 = open("EEMS_CMD_C1.mpt", 'r')
f2 = open("EEMS_CMD_C2.mpt", 'w')
for line in f1:
    f2.write(line.replace('"', ''))
f1.close()
f2.close()

f1 = open("EEMS_CMD_C2.mpt", 'r')
f2 = open("EEMS_CMD_C3.mpt", 'w')
for line in f1:
    f2.write(line.replace("'", ''))
f1.close()
f2.close()

f1 = open("EEMS_CMD_C3.mpt", 'r')
f2 = open("EEMS_CMD_C.mpt", 'w')
for line in f1:
    f2.write(line.replace("+", '&nbsp;'))
f1.close()
f2.close()

os.remove('EEMS_CMD_C1.mpt')
os.remove('EEMS_CMD_C2.mpt')
os.remove('EEMS_CMD_C3.mpt')

gdb_name = "EEMS_Model_C.gdb"
path = os.path.join(THIS_FOLDER, gdb_name)

try:
    arcpy.CreateFileGDB_management("./", gdb_name)
except:
    print("{} already exists".format(gdb_name))
    pass

out_featureclass = os.path.join(path, "EEMS_Model_C")
arcpy.CopyFeatures_management("EEMS_Model_C", out_featureclass)

zipf = zipfile.ZipFile('EEMS_Model_C.gdb.zip', 'w', zipfile.ZIP_DEFLATED)
zipdir("EEMS_Model_C.gdb/", zipf)
zipf.close()